# Проект: сервис для чтения книг

Цель: проанализировать базу данных сервиса для чтения книг по подписке.

## Описание данных

Таблица books  
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.    

Таблица authors  
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.    

Таблица publishers  
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства.  

Таблица ratings  
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.  

Таблица reviews  
Содержит данные о пользовательских обзорах на книги:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

## Подключение к базе данных

In [24]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
# '''Открываем соединение, получаем данные из sql, закрываем соединение'''
  with engine.connect() as con:
    return pd.read_sql(sql=sa.text(query), con = con)

## Исследование таблиц

### Таблица books

In [25]:
query = '''SELECT *
FROM books
LIMIT 5
'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [26]:
query = '''SELECT COUNT(*)
FROM books
'''
get_sql_data(query)

,count
0,1000


Всего у нас есть данные о 1000 книг.

### Таблица authors

In [27]:
query = '''SELECT *
FROM authors
LIMIT 5
'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [28]:
query = '''SELECT COUNT(*)
FROM authors
'''
get_sql_data(query)

,count
0,636


Всего представлено 636 авторов.

### Таблица publishers

In [29]:
query = '''SELECT *
FROM publishers
LIMIT 5
'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [30]:
query = '''SELECT COUNT(*)
FROM publishers
'''
get_sql_data(query)

,count
0,340


Всего представлено 340 издательств.

### Таблица ratings

In [31]:
query = '''SELECT *
FROM ratings
LIMIT 5
'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [32]:
query = '''SELECT COUNT(*)
FROM ratings
'''
get_sql_data(query)

,count
0,6456


Всего пользователи поставили 6456 оценок.

### Таблица reviews

In [33]:
query = '''SELECT *
FROM reviews
LIMIT 5
'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [34]:
query = '''SELECT COUNT(*)
FROM reviews
'''
get_sql_data(query)

,count
0,2793


Всего пользователи написали 2793 обзоров на книги.

## Задания

### Cколько книг вышло после 1 января 2000 года?

In [35]:
query = '''SELECT COUNT(DISTINCT book_id) 
FROM books 
WHERE publication_date::date > '2000-01-01'
'''
get_sql_data(query)

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку.

In [36]:
query = '''SELECT DISTINCT b.title, COUNT(DISTINCT r.review_id), AVG(ra.rating)
FROM books AS b
LEFT JOIN reviews AS r ON b.book_id=r.book_id
LEFT JOIN ratings AS ra ON b.book_id=ra.book_id
GROUP BY b.book_id
ORDER BY COUNT(DISTINCT r.review_id) DESC
'''
get_sql_data(query)

,title,count,avg
0,Twilight (Twilight #1),7,3.662500
1,Eat Pray Love,6,3.395833
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,Outlander (Outlander #1),6,4.125000
...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,Essential Tales and Poems,0,4.000000
997,Leonardo's Notebooks,0,4.000000
998,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000


Больше всего обзоров у книги "Сумерки" - 7 штук.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [37]:
query = '''WITH publ AS (SELECT p.publisher, COUNT(b.book_id) AS books_count
FROM books AS b
JOIN publishers AS p ON b.publisher_id=p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher)

SELECT publisher, books_count
FROM publ
ORDER BY books_count DESC
LIMIT 1
'''
get_sql_data(query)

,publisher,books_count
0,Penguin Books,42


Больше всего книг (42) выпустило издательство Penguin Books.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [59]:
query = '''SELECT a.author, AVG(r.rating)
FROM author AS a
JOIN books AS b ON b.author_id=a.author_id
JOIN ratings AS r ON b.book_id=r.book_id
WHERE b.book_id IN (SELECT book_id
                    FROM ratings 
                    GROUP BY book_id
                    HAVING COUNT(rating_id) >= 50
                    ORDER BY COUNT(rating_id) DESC)
GROUP BY a.author
ORDER BY AVG(r.rating) DESC
LIMIT 1
'''
get_sql_data(query)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг — Дж.К. Роулинг (средняя оценка: 4,28).

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [39]:
query = ''' WITH users_reviews AS (SELECT COUNT(review_id) AS total_reviews_amt, username
FROM reviews 
WHERE username IN (SELECT username
                FROM ratings
                GROUP BY username
                HAVING COUNT(rating_id) > 48)
                GROUP BY username)
SELECT AVG(total_reviews_amt)
FROM users_reviews
'''
get_sql_data(query)

,avg
0,24.0


В среднем пользователи, которые поставили больше 48 оценок, пишут 24 обзора.

## Выводы

После 1 января 2000 года вышло 819 книг.  
Больше всего обзоров у книги "Сумерки" - 7 штук.  
Больше всего книг (42) выпустило издательство Penguin Books.  
Автор с самой высокой средней оценкой книг — Дж.К. Роулинг (средняя оценка: 4,4).  
В среднем пользователи, которые поставили больше 48 оценок, пишут 24 обзора.